<a name="Data"></a>
### Data Prep

In [ ]:
!pip install -q gdown

In [ ]:
!pip install -q gdown

def download_from_drive(file_id, output_name):
  # Create a download link using the file ID
  url = f'https://drive.google.com/uc?id={file_id}'

  # Define the filename for the downloaded file
  output_file = f'{output_name}.csv'

  # Download the file using gdown
  !gdown $url -O $output_file

  # Verify that the file is downloaded
  !ls -lh $output_file

In [ ]:
# Download the masked dataset
download_from_drive('1z7TX6Qoa8pb4lqzKEgCAXPhulereOr-R', 'masked_dataset')

Downloading...
From: https://drive.google.com/uc?id=1z7TX6Qoa8pb4lqzKEgCAXPhulereOr-R
To: /content/masked_dataset.csv.csv
100% 10.9M/10.9M [00:00<00:00, 47.9MB/s]
-rw-r--r-- 1 root root 11M May  1 12:16 masked_dataset.csv.csv


In [ ]:
import pandas as pd

masked_dataset_df = pd.read_csv(
    '/content/masked_dataset.csv',
    sep='   ', header=None,
    on_bad_lines='skip',
    names=['response']
)

<ipython-input-6-391301afd016>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  masked_dataset_df = pd.read_csv(


In [ ]:
pd.set_option('display.max_colwidth', 500)
print(len(masked_dataset_df))
print(masked_dataset_df.loc[:5])

7998
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              response
0  "NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.On small-denomination, or ""consumer,"" CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Organisation, a Wilmington, Del., information service.On three-month ""consumer"" deposits

In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split


masked_dataset_df = masked_dataset_df.loc[:500] # Az adatbázis elemeinek csökkentése teljesítmény növelés érdekében
train_df, val_df = train_test_split(masked_dataset_df, train_size=0.9)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print(len(train_dataset))
print(len(val_dataset))

450
51


In [ ]:
import numpy as np
text_length = masked_dataset_df["response"].apply(lambda x: len(x.split()))
print(f"Average number of words per article: {np.round(text_length.mean())}")

Average number of words per article: 209.0


In [ ]:
system_message = 'Given a request to answer economic questions your answer must not contain sensitive information (like names of persons, name of organisations and PII information).'

In [ ]:
# train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n [/INST] ' + response for response in examples['response']]}, batched=True)
# val_dataset_mapped = val_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n [/INST] ' + response for response in examples['response']]}, batched=True)

train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + system_message.strip() + ' [/INST] ' + response for response in examples['response']]}, batched=True)
val_dataset_mapped = val_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + system_message.strip() + ' [/INST] ' + response for response in examples['response']]}, batched=True)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [ ]:
train_dataset_mapped["text"][5]

"[INST] <<SYS>>\nGiven a request to answer economic questions your answer must not contain sensitive information (like names of persons, name of organisations and PII information).\n<</SYS>>\n\nGiven a request to answer economic questions your answer must not contain sensitive information (like names of persons, name of organisations and PII information). [/INST] Person, a prolific deal maker in commercial real estate, announced a pair of multibillion-dollar transactions Monday, expanding his hotel empire and selling a large company that specializes in health-care properties.Mr. Person said he would sell his Organisation. to Organisation., a giant health-care real-estate investment trust, in a $2.6 billion cash-and-stock deal.Also Monday, Mr. Person's Organisation said it would acquire the Organisation, which owns 126 hotels in 35 U.S. states, from a real-estate fund sponsored by Organisation., for $1.92 billion.Separately, Mr. Person's flagship company, the publicly traded REIT Organi

## Train the model

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-7b-bnb-4bit", # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset_mapped,
    eval_dataset=val_dataset_mapped,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        per_device_eval_batch_size = 4,
        gradient_accumulation_steps = 1,
        gradient_checkpointing = True,
        max_grad_norm = 0.3,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        # num_train_epochs = 3,
        # max_steps = -1,
        warmup_ratio = 0.03,
        group_by_length = True,
        save_steps = 25,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/450 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/51 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.055 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 450 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 1
\        /    Total batch size = 4 | Total steps = 60
 "-____-"     Number of trainable parameters = 19,988,480


Step,Training Loss
5,2.434000
10,2.129500
15,1.796100
20,1.623300
25,1.547400
30,1.603000
35,1.649000
40,1.533300
45,1.486500
50,1.462200


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

258.8563 seconds used for training.
4.31 minutes used for training.
Peak reserved memory = 5.328 GB.
Peak reserved memory for training = 1.273 GB.
Peak reserved memory % of max memory = 36.127 %.
Peak reserved memory for training % of max memory = 8.632 %.


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
                    ["[INST] <<SYS>>\nGiven a request to explain certain concepts, please phrase your answers like you were talking to a 10 year old children.<</SYS>>\nPlease explain to me what large language models are and how they work![/INST]\n"],
                    return_tensors = "pt"
                    ).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = max_seq_length)

NameError: name 'FastLanguageModel' is not defined

In [ ]:
# Use the below function for gradio implementation
# def generate_response(message, history):
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
                    [f"[INST]<<SYS>>\n{system_message}<</SYS>>\nWrite an article about how the FED increased interest rates which affected major banks in particular JPMorgan.[/INST]\n"],
                    return_tensors = "pt"
                    ).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)
# return _

TBD: Implement Gradio interface

In [ ]:
# !pip install -q typer
# !pip install -q gradio

In [ ]:
# import gradio as gr
# gr.ChatInterface(generate_response).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://984018282c126074a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Measure the effectiveness of the fine-tuned model

In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.7 MB/s eta 0:00:00


In [ ]:
import re
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import nltk
nltk.download('names')
from nltk.corpus import names

from datasets import load_dataset
conll = load_dataset("conll2003")

import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
# Download the people dataset
download_from_drive('1eQAfPu9UrgEuGLt47w7pcGZvcVXXrVET', 'people')

# Download the organisation dataset
download_from_drive('1tQgFcjNwrpscNZufK2wsqC-V7j3jUoTC', 'organisations')

Downloading...
From (original): https://drive.google.com/uc?id=1eQAfPu9UrgEuGLt47w7pcGZvcVXXrVET
From (redirected): https://drive.google.com/uc?id=1eQAfPu9UrgEuGLt47w7pcGZvcVXXrVET&confirm=t&uuid=1c548f49-54a8-4801-a0bd-234c029832fc
To: /content/people.csv
100% 475M/475M [00:09<00:00, 48.1MB/s]
-rw-r--r-- 1 root root 453M May  2 12:10 people.csv
Downloading...
From (original): https://drive.google.com/uc?id=1tQgFcjNwrpscNZufK2wsqC-V7j3jUoTC
From (redirected): https://drive.google.com/uc?id=1tQgFcjNwrpscNZufK2wsqC-V7j3jUoTC&confirm=t&uuid=64edac4c-ca39-42f0-93fc-cd84abf9e67c
To: /content/organisations.csv
100% 1.72G/1.72G [00:25<00:00, 67.1MB/s]
-rw-r--r-- 1 root root 1.6G May  2 12:10 organisations.csv


In [ ]:
# OPTION 1
# Load the datsets into a df and explore them
crunchbase_people = pd.read_csv('/content/people.csv', sep=',', on_bad_lines='skip')
crunchbase_organisations = pd.read_csv('/content/organisations.csv', sep=',', on_bad_lines='skip')

In [ ]:
crunchbase_people.head()

,uuid,type,first_name,last_name,cb_url,logo_url,facebook_url,twitter_url,linkedin_url,city,region,country_code,featured_job_title,featured_job_organization_name,featured_job_organization_uuid
0,ed13cd36-fe2b-3707-197b-0c2d56e37a71,person,Ben,Elowitz,https://www.crunchbase.com/person/ben-elowitz?...,https://images.crunchbase.com/image/upload/t_c...,http://www.facebook.com/elowitz,http://twitter.com/elowitz,https://www.linkedin.com/in/elowitz,Seattle,Washington,USA,Co-Founder,Zeitworks,1d845b32-7d80-47af-957d-78ccbeeaefb6
1,5ceca97b-493c-1446-6249-5aaa33464763,person,Kevin,Flaherty,https://www.crunchbase.com/person/kevin-flaher...,https://images.crunchbase.com/image/upload/t_c...,NaN,https://twitter.com/tallkp,http://www.linkedin.com/in/kevinflaherty,Mercer Island,Washington,USA,Team Member,DRSmedia,789e5e4d-0c90-d06e-92a0-b800b461c3da
2,9f99a98a-aa97-b30b-0d36-db67c1d277e0,person,Raju,Vegesna,https://www.crunchbase.com/person/raju-vegesna...,https://images.crunchbase.com/image/upload/t_c...,NaN,https://twitter.com/rajuv,https://www.linkedin.com/in/raju-vegesna-78a6541,San Francisco,California,USA,Chief Evangelist,Zoho,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7
3,6e1bca72-a865-b518-b305-31214ce2d1b0,person,Ian,Wenig,https://www.crunchbase.com/person/ian-wenig?ut...,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,NaN,NaN,NaN,NaN,VP Business Development,Zoho,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7
4,80d25c23-9726-9dda-5852-39cdf4810ea5,person,Ron,Gorodetzky,https://www.crunchbase.com/person/ron-gorodetz...,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,NaN,NaN,NaN,NaN,Co-Founder and CTO,Fflick,aa3bf156-06af-5b6e-215d-9e7211fc173b


In [ ]:
crunchbase_people.describe()

,uuid,type,first_name,last_name,cb_url,logo_url,facebook_url,twitter_url,linkedin_url,city,region,country_code,featured_job_title,featured_job_organization_name,featured_job_organization_uuid
count,1222256,1222256,1222243,1222181,1222239,961423,170876,223817,1028253,951565,989845,1055665,1207196,1207220,1207222
unique,1222256,1,126037,364302,1222239,961404,169971,222971,1027712,24695,1811,209,248141,629760,646212
top,ed13cd36-fe2b-3707-197b-0c2d56e37a71,person,David,Smith,https://www.crunchbase.com/person/ben-elowitz?...,https://images.crunchbase.com/image/upload/t_c...,https://www.facebook.com/profile.php,https://twitter.com/,https://www.linkedin.com/in/,New York,California,USA,Founder,Google,6acfa7da-1dbd-936e-d985-cf07a1b27711
freq,1,1222256,19726,4473,1,3,65,34,14,50803,126277,532004,67043,2956,2956


In [ ]:
crunchbase_organisations.head(30)

,uuid,name,type,primary_role,cb_url,domain,homepage_url,logo_url,facebook_url,twitter_url,linkedin_url,combined_stock_symbols,city,region,country_code,short_description
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,company,https://www.crunchbase.com/organization/wetpai...,wetpaint.com,http://www.wetpaint.com/,https://images.crunchbase.com/image/upload/t_c...,https://www.facebook.com/Wetpaint,https://twitter.com/wetpainttv,https://www.linkedin.com/company/recruitment-c...,NaN,New York,New York,USA,Wetpaint offers an online social publishing pl...
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,company,https://www.crunchbase.com/organization/zoho?u...,zoho.com,https://www.zoho.com,https://images.crunchbase.com/image/upload/t_c...,http://www.facebook.com/zoho,http://twitter.com/zoho,https://www.linkedin.com/company/zoho,NaN,Pleasanton,California,USA,"Zoho offers a suite of business, collaboration..."
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,company,https://www.crunchbase.com/organization/digg?u...,digg.com,http://www.digg.com,https://images.crunchbase.com/image/upload/t_c...,http://www.facebook.com/digg,http://twitter.com/digg,http://www.linkedin.com/company/digg,NaN,New York,New York,USA,Digg Inc. operates a website that enables its ...
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,investor,https://www.crunchbase.com/organization/omidya...,omidyar.com,http://www.omidyar.com,https://images.crunchbase.com/image/upload/t_c...,http://www.facebook.com/OmidyarNetwork,http://twitter.com/OmidyarNetwork,http://www.linkedin.com/company/22806,NaN,Redwood City,California,USA,Omidyar Network is an investment firm.
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta,organization,company,https://www.crunchbase.com/organization/facebo...,meta.com,https://meta.com,https://images.crunchbase.com/image/upload/t_c...,https://www.facebook.com/Meta,https://www.twitter.com/Meta,https://www.linkedin.com/company/meta,nasdaq:META,Menlo Park,California,USA,Meta is a social technology company that enabl...
5,7ca12f7a-2f8e-48b4-a8d1-1a33a0e275b9,Trinity Ventures,organization,investor,https://www.crunchbase.com/organization/trinit...,trinityventures.com,http://www.trinityventures.com,https://images.crunchbase.com/image/upload/t_c...,http://www.facebook.com/pages/Menlo-Park-CA/Tr...,http://twitter.com/trinityventures,http://www.linkedin.com/company/trinity-ventures,NaN,Menlo Park,California,USA,Trinity Ventures is an early stage venture cap...
6,b08efc27-da40-505a-6f9d-c9e14247bf36,Accel,organization,investor,https://www.crunchbase.com/organization/accel?...,accel.com,http://www.accel.com,https://images.crunchbase.com/image/upload/t_c...,NaN,http://www.twitter.com/accel,https://www.linkedin.com/company/accel-vc,NaN,Palo Alto,California,USA,Accel is an seed and growth-stage venture capi...
7,8d5c7e48-82da-3025-dd46-346a31bab86f,Meritech Capital Partners,organization,investor,https://www.crunchbase.com/organization/merite...,meritechcapital.com,http://www.meritechcapital.com,https://images.crunchbase.com/image/upload/t_c...,http://www.facebook.com/MeritechCapital,http://twitter.com/MeritechCapital,https://www.linkedin.com/company/meritech-capi...,NaN,Palo Alto,California,USA,Meritech Capital provides late-stage venture c...
8,60485007-8856-bbac-aa1b-c535c41f5f47,Omnidrive,organization,company,https://www.crunchbase.com/organization/omnidr...,omnidrive.com,http://www.omnidrive.com,https://images.crunchbase.com/image/upload/t_c...,http://www.facebook.com/Nomadesk,http://twitter.com/Nomadesk,NaN,NaN,Palo Alto,California,USA,Omnidrive provides online storage facilities t...
9,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,organization,company,https://www.crunchbase.com/organization/geni?u...,geni.com,http://www.geni.com,https://images.crunchbase.com/image/upload/t_c...,https://www.facebook.com/genidotcom/,http://twitter.com/geni,https://www.linkedin.com/company/geni-com/about/,NaN,West Hollywood,California,USA,Geni is

In [ ]:
crunchbase_organisations.describe()

,uuid,name,type,primary_role,cb_url,domain,homepage_url,logo_url,facebook_url,twitter_url,linkedin_url,combined_stock_symbols,city,region,country_code,short_description
count,3231431,3231428,3231431,3231431,3231427,3231431,3231431,3093516,1991895,1418546,2512997,47647,3076370,3076372,3076372,3231411
unique,3231431,3069914,1,3,3231427,3213463,3228018,3093393,1981782,1410981,2497422,47094,73428,2389,225,3221689
top,e1393508-30ea-8a36-3f96-dd3226033abd,Spark,organization,company,https://www.crunchbase.com/organization/wetpai...,heylink.me,https://korek.bio/idgacornagamas69,https://images.crunchbase.com/image/upload/t_c...,https://www.facebook.com/profile.php,https://twitter.com/,https://www.linkedin.com/company/studio-dentis...,bkk:BKK,London,California,USA,SaaS
freq,1,51,3231431,3182327,1,237,71,3,730,86,408,18,73255,238419,1308786,526


In [ ]:
def remove_punctuation(text):
  # Define a pattern to match characters that are not alphanumeric or whitespace
  pattern = "[^a-zA-Z0-9 ]"
  # Substitute punctuation characters with an empty string
  text = re.sub(pattern, "", text)
  return text

In [ ]:
def remove_stopwords(token_list):
  """
  This function removes stopwords from a list of tokens.
  """
  # Get stopwords in your desired language (e.g., English)
  stop_words = set(stopwords.words('english'))

  # Filter out stopwords from the token list
  filtered_tokens = [str(token).lower() for token in token_list if str(token).lower() not in stop_words]

  return filtered_tokens

In [ ]:
# Get a distinct list of names from the databases
people_names = list(set(list(crunchbase_people.first_name) + list(crunchbase_people.last_name)))

In [ ]:
org_names = list(set(crunchbase_organisations.name))[:400000] # Limit the number of organisational names as the list is too large
sensitive_tokens = list(set(people_names + org_names))
sensitive_tokens = [remove_punctuation(str(sublist).lower()) for sublist in sensitive_tokens] # Creates a list of ngrams
# org_names = [str(word).lower() for sublist in org_names for word in remove_punctuation(str(sublist)).split()] # Creates a list of unigrams
print(sensitive_tokens[:100])

['garrett hewitt international', 'cornerstone treatment facilities network', 'westcoast hardwoods', 'australian hand therapy association', 'neon flux', 'clement cambare', 'tokbird', 'photogurus', 'transfermycarcom', 'kheprion', 'music center', 'ultraorganics', 'metrominas', 'center helene borel', 'faerm', 'arnechristian', 'powney', 'lynx finance', 'simple programmer', 'shoelace learning', 'appfocus', 'koolwijk polsbroek', 'john hartvig', 'talbi', 'jabberyak', 'korczynska', 'chepesiuk', 'mendoza evoli', 'vitalize interiors', 'grosu', 'inflammation research association', 'costas indianos  co', 'khanduja', 'washidzu', 'innowi', 'agbonaye', 'huberfeely', 'pink door fabrics', 'gonthiez frres', 'rter', 'evans audio visual', 'shuilong', 'glassflow', 'frice consulting', 'codeobjects', 'imagine ips', 'mokoto', 'valasek', 'gorritz', 'tripsay', 'univerbal formerly quazel', 'babak hassan', 'inspur international', 'blueprint media', 'sorampuri', 'hongsun', 'central de tratamento de resduos de minas

In [ ]:
print(f"Length of list containing people's names: {len(people_names)}")
print(f"Length of list containing organisations' names: {len(org_names)}")
print(f"Length of list containing all sensitive names: {len(sensitive_tokens)}")

Length of list containing people's names: 467045
Length of list containing organisations' names: 400000
Length of list containing all sensitive names: 862482


In [ ]:
# Remove sensitive tokens that are stopwords
sensitive_tokens = remove_stopwords(sensitive_tokens)
print(f"List without stopwords: {len(sensitive_tokens)}")

List without stopwords: 862265


In [ ]:
sensitive_tokens = [tokens for tokens in sensitive_tokens if len(tokens) > 2]
print(f"List with longer than 2-letter words: {len(sensitive_tokens)}")

List with longer than 2-letter words: 860565


In [ ]:
def stem_text(text):
  # Initialize the stemmer
  stemmer = PorterStemmer()
  # Tokenize the text (split into words)
  tokens = nltk.word_tokenize(text)
  # Apply stemming to each token
  stemmed_words = [stemmer.stem(word) for word in tokens]
  # Join the stemmed words back into a string (optional)
  return " ".join(stemmed_words)

In [ ]:
# Remove tokens that are not sensitive
# sensitive_tokens.remove("organisation")
sensitive_tokens.remove("april")
sensitive_tokens.remove("may")
sensitive_tokens.remove("june")
sensitive_tokens.remove("august")
# sensitive_tokens.remove("")
# sensitive_tokens.remove("5") # Should remove numbers with regex

ValueError: list.remove(x): x not in list

In [ ]:
sensitive_tokens.remove("june")

ValueError: list.remove(x): x not in list

In [ ]:
assert "fed" in sensitive_tokens

In [ ]:
assert "organisation" not in sensitive_tokens
assert "june" not in sensitive_tokens
assert "5" not in sensitive_tokens
assert "" not in sensitive_tokens

In [ ]:
# Decomissioned: Only useful if the list sensitive_tokens consists of 1-word long tokens
def calc_metric(text):
  """
  Calculates the ratio of non-sensitive words within a text
  """

  nominator = len(text.split())
  numerator_words = [str(word) for word in text.split() if str(word).lower() not in sensitive_tokens]
  numerator = len(numerator_words)
  ratio = numerator / nominator if nominator > 0 else 0
  print(f"Numerator: {numerator}\nNominator: {nominator}\nNumerator words: {numerator_words}")
  return ratio

In [ ]:
# The text (previously generated by the fine-tuned model) has 3 sensitive words (Fed) out of the 175 (that is 1.7%).
text = "The Organisation's monetary policy committee raised the benchmark interest rate by 0.25% to 1.75% at its June meeting, the second such increase in 2018. The hike was widely expected and was the first since December 2016. The move was largely anticipated and follows the Fed's decision in March to increase rates for the first time in two years.The move is likely to have a bigger impact on the Organisation than on the Organisation. The Organisation's key rate is still at 1.25%, and the central bank has signaled that it will continue to hold rates steady for the foreseeable future. That's in contrast to the Fed, which is widely expected to raise rates three more times this year, and two more times in 2019.The Organisation's stock market is also likely to be less affected by the hike than the Organisation's. The Organisation's stock market is down about 2% since the Fed's March rate hike. The Organisation's stock market is down about 5% since the Fed's March rate hike.The Organisation's stock market is down about 2% since"
punct_free_text = remove_punctuation(text)
print(f"Without punctutation: {punct_free_text}")
precision = calc_metric(punct_free_text)

# Avoid stemming as it does not seem to be reliable

# stemmed_text = stem_text(punct_free_text)
# print(f"Lemmatized text: {stemmed_text}")
# precision = calc_metric(stemmed_text)
print(precision)

Without punctutation: The Organisations monetary policy committee raised the benchmark interest rate by 025 to 175 at its June meeting the second such increase in 2018 The hike was widely expected and was the first since December 2016 The move was largely anticipated and follows the Feds decision in March to increase rates for the first time in two yearsThe move is likely to have a bigger impact on the Organisation than on the Organisation The Organisations key rate is still at 125 and the central bank has signaled that it will continue to hold rates steady for the foreseeable future Thats in contrast to the Fed which is widely expected to raise rates three more times this year and two more times in 2019The Organisations stock market is also likely to be less affected by the hike than the Organisations The Organisations stock market is down about 2 since the Feds March rate hike The Organisations stock market is down about 5 since the Feds March rate hikeThe Organisations stock market 

In [ ]:
def check_sensitive_tokens_percentage(text, sensitive_tokens):
  text_tokens = text.split()
  regex = r"\b(" + "|".join(sensitive_tokens) + r")\b"  # Join sensitive tokens with OR operator
  # print(regex)
  matches = len(re.findall(regex, text))  # Count matches using findall
  print(f"Length of sensitive words: {matches}")
  percentage = round(matches / len(text_tokens), 4)
  print(f"Length of text: {len(text_tokens)}")
  print(f"The text's {percentage * 100}% tokens are sensitive.")
  return (percentage)

In [ ]:
check_sensitive_tokens_percentage(punct_free_text, sensitive_tokens[:500000])

Length of sensitive words: 22
Length of text: 175
The text's 12.57% tokens are sensitive.


0.1257

In [ ]:
# OPTION 2

# Get People's names from the NLTK library
male_names = list(set(names.words('male.txt')))
female_names = list(set(names.words('female.txt')))
people_names_nltk = male_names + female_names
people_names_nltk = [name.lower() for name in people_names_nltk]
print(f'Number of male names: {len(male_names)}')
print(f'Number of female names: {len(female_names)}')
print(f'Number of names in total: {len(people_names_nltk)}')
print(f'Example names: {people_names_nltk[:15]}')

Number of male names: 2943
Number of female names: 5001
Number of names in total: 7944
Example names: ['morlee', 'roice', 'chevalier', 'nathanial', 'mohammed', 'west', 'constantine', 'oswell', 'ulysses', 'sarge', 'thayne', 'giavani', 'humbert', 'ave', 'abelard']


In [ ]:
from datasets import concatenate_datasets

def process_dataset(conll_dataset, ner_tag_number):
  """
  Aim of this function is to process the conll dataset. The following processes are done:
  - train-test-val sets are combined
  - only token and ner_tags columns are kept
  - tokens are filtered based on ner_tags given as an input parameter
  - distinct list of tokens are returned
  """
  # Combine the train-test-val sets
  combined_dataset = concatenate_datasets([conll_dataset['train'], conll_dataset['validation'], conll_dataset['test']])

  # Only keep the token and ner_tags columns
  combined_dataset = combined_dataset.remove_columns(['id', 'pos_tags', 'chunk_tags'])

  # Define a function to filter tokens based on ner_tags and make tokens lower-cased
  def filter_tokens_by_ner_tags(row):
      tokens = row['tokens']
      ner_tags = row['ner_tags']
      selected_tokens = [token.lower() for token, tag in zip(tokens, ner_tags) if tag in ner_tag_number]
      selected_ners = [tag for token, tag in zip(tokens, ner_tags) if tag in ner_tag_number]
      return {'tokens': selected_tokens, 'ner_tags': selected_ners}

  # Apply the filter function to each row of the dataset
  filtered_tokens_dataset = combined_dataset.map(filter_tokens_by_ner_tags)
  print(f"Filtered dataset: {filtered_tokens_dataset} \n")
  print(f"Example data: {filtered_tokens_dataset[:5]} \n")

  # Access the tokens and ner_tags directly from the filtered dataset
  all_tokens = filtered_tokens_dataset['tokens']
  all_flattened_tokens = [token for sublist in all_tokens for token in sublist]  # Flatten the list of lists


  # Make the list distinct
  tokens_list = list(set(all_flattened_tokens))
  print(f"Number of tokens in the list of tokens: {len(tokens_list)} \n")
  print(f"List of tokens: {tokens_list[:15]} \n")

  return tokens_list

In [ ]:
# Get the tokens for Organisations' names from the conll dataset
org_names = process_dataset(conll, [3, 4])

# Get the tokens for People's names from the conll dataset
people_names_conll = process_dataset(conll, [1, 2])

# Combine list of tokens for every sensitive names (conll and nltk)
sensitive_tokens = list(set(people_names_nltk + people_names_conll + org_names))
print(len(sensitive_tokens))

Map:   0%|          | 0/20744 [00:00<?, ? examples/s]

Filtered dataset: Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 20744
}) 

Example data: {'tokens': [['eu'], [], [], ['european', 'commission'], ['european', 'union']], 'ner_tags': [[3], [], [], [3, 4], [3, 4]]} 

Number of tokens in the list of tokens: 3005 

List of tokens: ['psi', 'kay', 'administration', 'essilor', 'babel', 'ross', 'impala', 'holding', 'nihon', 'command', 'across', 'boland', 'nld', 'zhemchuzhina', 'plzen'] 



Map:   0%|          | 0/20744 [00:00<?, ? examples/s]

Filtered dataset: Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 20744
}) 

Example data: {'tokens': [[], ['peter', 'blackburn'], [], [], ['werner', 'zwingmann']], 'ner_tags': [[], [1, 2], [], [], [1, 2]]} 

Number of tokens in the list of tokens: 5404 

List of tokens: ['schomberg', 'lamy', 'feugill', 'tauziat', 'corretja', 'wilson', 'votava', 'ridgeon', 'janez', 'voica', 'carling', 'felicia', 'kamil', 'handrik', 'fung'] 

14499


In [ ]:
# Sample text
text = "This is a sample text. There are punctuations! Let's remove them and apply stemming."

# Remove punctuation
text_without_punctuation = remove_punctuation(text)

# Apply stemming
stemmed_text = stem_text(text_without_punctuation)

# Print the results
print("Original Text:", text)
print("Text without Punctuation:", text_without_punctuation)
print("Stemmed Text:", stemmed_text)

Original Text: This is a sample text. There are punctuations! Let's remove them and apply stemming.
Text without Punctuation: This is a sample text There are punctuations Lets remove them and apply stemming
Stemmed Text: thi is a sampl text there are punctuat let remov them and appli stem


<a name="Save"></a>
## Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

NameError: name 'model' is not defined

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)